<b>Introduction</b> <br>
One day, we’ll have big fault tolerant computers. They’ll work flawlessly, automatically correcting any errors that occur. But, unfortunately, this isn’t the situation at the moment. 

Today’s quantum computers are relatively small and, more importantly, noisy. This means that we need to think carefully about their noise and how to work around it. One technique for doing this is <i>zero-noise extrapolation</i> or <i>ZNE</i>.

<b>What is ZNE?</b> <br>
ZNE was first introduced in 2017 <a href="https://arxiv.org/abs/1612.02058">[1]</a> <a href="https://journals.aps.org/prx/abstract/10.1103/PhysRevX.7.021050">[2]</a>. It’s based on the idea of systematically <i>adding extra noise</i> to a circuit. After doing this, we measure the expectation value of some observable and, finally, extrapolate back to the ideal case with zero noise. 

At first glance, ZNE seems like a strange idea. It involves emulating a noiseless quantum computer by adding more noise. However, the key idea is that we add noise in a systematic way. This allows us to learn the system's noise behaviour. Next, we use this knowledge to extrapolate to the behaviour of an ideal, noiseless quantum computer.

<b>Why should you care about ZNE?</b> <br>
ZNE is incredibly important as it allows us to efficiently determine the expectation values that a perfect, error-free quantum computer would have measured. But, using one of today’s imperfect error-prone quantum computers. 

This is important it opens the door to doing something useful on today's quantum computers. More specifically, to demonstrating something called <a href="https://research.ibm.com/blog/what-is-quantum-utlity">quantum utility.</a>

Quantum utility is the ability of a quantum computer to perform calculation that no brute-force classical calculation can reproduce (in a reasonable amount of time). Note that this does <i>not</i> mean that it’s impossible to come up with a specifically tailored, sophisticated classical algorithm that can reproduce the result. But, instead, that it’s impossible for a general-purpose algorithm to do it.

Quantum utility is a key milestone in the experimental realization of quantum computers and quantum advantage.

<b>How can I implement ZNE?</b> <br>
<h2 style="font-size: value;">
The details of ZNE are complicated. So, you might think that you need to be an expert on quantum computers—and quantum physics—to implement it in your quantum programs. But, nothing could be further from the truth.<br> 
<h2 style="font-size: value;">
Using some of Qiskit’s new features, you can implement it in just a single line of code.
<h2 style="font-size: value;">
Here it is:<br><br>
<i>options.resilience_level = 2</i>
<h2 style="font-size: value;">
That’s all there is to it.
</h2>
<a href="https://docs.quantum.ibm.com/run/configure-error-mitigation">Source: Qiskit documentation on error mitigation</a>

To provide more context, here’s a fully fleshed out code example. It’s based on a simple quantum circuit that consists of ten X gates in a row:
<img src="ten_x_gates_circuit.png" alt="Drawing" style="width: 900px;"/>

First, let's simulate running the circuit on an ideal, noiseless quantum computer. After implementing the circuit, we'll measure the expectation value of Z.

In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator, Options, Session
from qiskit import QuantumCircuit
from qiskit_aer.noise import NoiseModel
from qiskit.quantum_info import SparsePauliOp
from qiskit_aer.noise import depolarizing_error

#create a model for the noise in the system
noise_model = NoiseModel()

#probability of an error occurring
#Initially, we'll consider an ideal noiseless system.
p_error = 0.0
error = depolarizing_error(p_error, 1)
noise_model.add_quantum_error(error, ['x'], [0])

backend = "ibmq_qasm_simulator"

#If you haven't already saved your IBM account, you need to include the line below.
#Note: Change "MY_IBM_QUANTUM_TOKEN" to your API token
#You can find this token by logging in here: https://quantum.ibm.com/
#QiskitRuntimeService.save_account(channel="ibm_quantum", token="MY_IBM_QUANTUM_TOKEN")

service = QiskitRuntimeService()
options = Options()

options.simulator = {
    "noise_model": noise_model,
    "seed_simulator": 52
}

options.resilience_level = 0
options.optimization_level = 0

#Define a quantum circuit that implements ten X gates in a row.
qc = QuantumCircuit(1,1)
for i in range(10):
    qc.x(0)
    qc.barrier(0)

#define the Z operator that we want to measure the expectation value of at the end
Z = SparsePauliOp.from_list([("Z", 1)])

with Session(service=service, backend=backend):
    #calculate the expectation value of Z in the ideal noiseless circuit
    estimator = Estimator(options=options)
    job = estimator.run(
        circuits=qc,
        observables=[Z]
    )
    
    result = job.result()
    exp_value_ideal = result.values[0]
 
    print(f" > IDEAL NOISELESS CIRCUIT: expectation value of Z = {exp_value_ideal:.5f}")   

 > IDEAL NOISELESS CIRCUIT: expectation value of Z = 1.00000


The result makes sense as ten X gates in a row is equivalent to the identity operation. Now, let's add some noise to the system and calculate the expectation value of Z again.

In [2]:
#probability of an error occurring
p_error = 0.01
error = depolarizing_error(p_error, 1)
noise_model.add_quantum_error(error, ['x'], [0])

options.simulator = {
    "noise_model": noise_model,
    "seed_simulator": 52
}

#This corresponds to *not* using ZNE.
options.resilience_level = 0
options.optimization_level = 0

with Session(service=service, backend=backend):
    #calculate the expectation value of Z in a noisy circuit without using ZNE
    estimator = Estimator(options=options)
    job = estimator.run(
        circuits=qc,
        observables=[Z]
    )
    
    result = job.result()
    exp_value = result.values[0]
 
    print(f" > IDEAL NOISELESS CIRCUIT: expectation value of Z = {exp_value_ideal:.5f}")
    print(f" > NOISY CIRCUIT WITHOUT ZNE: expectation value of Z = {exp_value:.5f}")   

 > IDEAL NOISELESS CIRCUIT: expectation value of Z = 1.00000
 > NOISY CIRCUIT WITHOUT ZNE: expectation value of Z = 0.90150


Notice how the expectation value decreased due to the presence of the noise. Let's see if we can use ZNE to get a better estimate of the expectation value.

In [3]:
#The two lines below implement ZNE. That's all there is to it.
#Note that there are different types of ZNE.
#'LocalFoldingAmplifier' specifies one type. For more details on this, see https://docs.quantum.ibm.com/run/configure-error-mitigation
options.resilience_level = 2
options.resilience.noise_amplifier = 'LocalFoldingAmplifier'

with Session(service=service, backend=backend):
    #calculate the expectation value of Z in a noisy circuit using ZNE
    estimator = Estimator(options=options)
    job = estimator.run(
        circuits=qc,
        observables=[Z]
    )
    result = job.result()
    exp_value_with_zne = result.values[0]
 
    print(f" > IDEAL NOISELESS CIRCUIT: expectation value of Z = {exp_value_ideal:.5f}")
    print(f" > NOISY CIRCUIT WITHOUT ZNE: expectation value of Z = {exp_value:.5f}") 
    print(f" > NOISY CIRCUIT WITH ZNE: expectation value of Z = {exp_value_with_zne:.5f}")

 > IDEAL NOISELESS CIRCUIT: expectation value of Z = 1.00000
 > NOISY CIRCUIT WITHOUT ZNE: expectation value of Z = 0.90150
 > NOISY CIRCUIT WITH ZNE: expectation value of Z = 0.97496


Notice how the noisy expectation value of Z with ZNE is closer to the ideal case. This illustrates the usefulness of ZNE.

<b>When is ZNE good to use?</b> <br>
Whenever you want to calculate the expectation value of an observable on a noisy quantum computer and would like to get a good estimate of what its value woudld be on an ideal noiseless quantum computer.

<b>More details on ZNE (optional)</b> <br>

Consider an ideal quantum circuit that implements the unitary operation $U$. Imagine that we want to implement the circuit and then measure the expectation value $\langle O \rangle$ of some observable $O$. 

<img src="basic_circuit_just_U.png" alt="Drawing" style="width: 400px;"/>

If we try doing this on a real quantum computer, we'll only implement a noisy version of the circuit. So, our measured value for $\langle O \rangle$ will differ from the ideal value.

We can use ZNE to estimate the ideal noiseless value of $\langle O \rangle$ as follows: </br> </br>
<b>STEP 1</b> <br>
Implement the operation $U U^{\dagger} U$ on the circuit.<br> 
Measure $\langle O \rangle$.<br>
(Note that, without any noise, $U^{\dagger} U = \mathbb{I}$ and so $U U^{\dagger} U = U$.)<br>
Physically, $U U^{\dagger} U$ corresponds to increasing the amount of noise in the circuit by implementing $U$, then implementing its time reversal (as $U^{\dagger} = U^{-1}$), and then implementing $U$ again.

<img src="circuit_scale_factor_3.png" alt="Drawing" style="width: 400px;"/>

<b>STEP 2</b> <br>
Increase the noise even more by implementing $U \left( U^{\dagger} U \right)^{2}$.<br>
Measure $\langle O \rangle$.

<img src="circuit_scale_factor_5.png" alt="Drawing" style="width: 400px;"/>

<b>STEP 3</b> <br>
Increase the noise a few more times by implementing $U \left( U^{\dagger} U \right)^{n}$, where $n=3,4,5...$.<br>
Each time, we measure $\langle O \rangle$ at the end.

<b>STEP 4</b> <br>
Use all the measured expectation values to extrapolate and calculate what $\langle O \rangle$ would be in an ideal, noiseless system: $\langle O \rangle_{\rm ideal}$.

<img src="extrapolation.png" alt="Drawing" style="width: 400px;"/>

In practice, we can add noise in a more fine-grained way by only implementing some of the gates in the quantum circuit. Let $U$ be made up of $d$ quantum gates: 
$U = L_{1} L_{2} ... L_{d}$,
where $L_{i}$ is the $i^{\rm th}$ gate.

We can add noise in a more controlled way by implementing the following sequence:<br>
$U \left( U^{\dagger} U \right)^{n} L_{1} L_{2} ... L_{s} L_{s}^{\dagger} ... L_{2}^{\dagger} L_{1}^{\dagger}$,
where $1 \leq s \leq d$.

Physically, this corresponds to just implementing some of the gates and then implementing their time reversals.

<img src="circuit_partial_fold.png" alt="Drawing" style="width: 400px;"/>

In ZNE, we characterize the amount of noise that we add by the parameter $\lambda = 1+ 2\left( n + s/d \right)$ which is called the <i>scale factor</i>.

Note: This section describes ZNE with global folding, one of the many types of ZNE.<br><br> 
To learn more about ZNE, <a href="https://arxiv.org/abs/2005.10921v1">this paper</a> provides an excellent overview. 